In [1]:
import pandas as pd
import sqlite3

# Path to your CSV file
csv_file_path = 'historical-futures-data.csv'
# Path to your SQLite database file
db_file_path = 'C:/Users/ftelias/OneDrive/Documents/GitHub/humquote/futures_prices.db'
# Name of the table to which the data will be appended
table_name = 'futures_data'

def create_connection(db_file):
    """Create and return a connection to the database."""
    try:
        conn = sqlite3.connect(db_file)
        print("Connection established")
        return conn
    except sqlite3.Error as e:
        print(f"Error connecting to database: {e}")
    return None

# Changed this function to the ASX-futures script so append to database is directly from csv file
#def pivot_dataframe(df):
#    """Pivot the DataFrame to wide format."""
#    df_pivot = df.pivot_table(index=['Quote Date', 'Year'], columns='State', values='Price', aggfunc='first').reset_index()
#    return df_pivot

def append_csv_to_database(csv_file, db_file, table):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file, date_format='%Y-%m-%d')
    
    # Convert 'Quote Date' to datetime and sort in descending order
    df['Quote Date'] = pd.to_datetime(df['Quote Date'], format="%Y/%m/%d")

    df_sorted = df.sort_values(by='Quote Date', ascending=False)
    
    #df_wide = pivot_dataframe(df)
    
    #df_sorted = df_wide.sort_values(by='Quote Date', ascending=False)
    
    conn = create_connection(db_file)
    if conn is not None:
        cursor = conn.cursor()
        
        for index, row in df_sorted.iterrows():
            quote_date = row['Quote Date'].strftime('%Y-%m-%d')  # Format to "yyyy-mm-dd"
            year = int(row['Year'])
            query = f"SELECT COUNT(*) FROM {table} WHERE `Quote Date` = ? AND `Year` = ? ORDER BY `Quote Date` DESC"
            cursor.execute(query, (quote_date, year))
            exists = cursor.fetchone()[0]

            if exists == 0:
                # Convert row to DataFrame and ensure correct data types
                row_df = pd.DataFrame(row).transpose()
                row_df['Quote Date'] = pd.to_datetime(row_df['Quote Date']).dt.date
                row_df['Year'] = row_df['Year'].astype(int)
                
                row_df.to_sql(table, conn, if_exists='append', index=False)
                print(f"Appended data for {quote_date}, Year {year} to the database.")
            else:
                print(f"Skipped data for {quote_date}, Year {year} (already exists).")
        
        conn.close()
        print("Connection closed")
    else:
        print("Failed to connect to the database.")

append_csv_to_database(csv_file_path, db_file_path, table_name)

ValueError: time data "2024-06-13" doesn't match format "%d/%m/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.